In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# データ読み込み
notepc_data = pickle.load(open("notepc_data.pkl", "br"))

# カラム
# 0: price
# 1: num_of_cores
# 2: clock
# 3: memory
# 4: disk
# 5: weight
# 6: months

In [ ]:
# 実際に使うデータを切り出し
# 価格は 1/100000スケールにて。価格もクロック数も、（ベクトルのままではなく）行列形式に変換
price = np.array([t[0] for t in notepc_data], dtype=np.float32) / 100000.
price.resize((price.size, 1))
clock = np.array([t[2] for t in notepc_data], dtype=np.float32)
clock.resize((clock.size, 1))

In [ ]:
# データ可視化
plt.scatter(clock, price)
plt.xlabel("clock")
plt.ylabel("price")

In [ ]:
# シグモイド関数を用意
def sigmoid(x):
    x = np.clip(x, -700, 700)
    return 1. / (1 + np.exp(-x))

In [ ]:
# モデルを用意
def layer1(x, w1, b1):
    # 1層目
    a1 = np.dot(x, w1) + b1
    return sigmoid(a1)

def layer2(x, w2, b2):
    # 2層目
    a2 = np.dot(x, w2.transpose()) + b2
    return a2

def model(x, w1, b1, w2, b2):
    # 1層目
    h1 = layer1(x, w1, b1)
    
    # 2層目
    y = layer2(h1, w2, b2)
    
    return y

In [ ]:
# モデルのプロット用関数
x_val = np.linspace(1., 3.2, 101)
x_val.resize((x_val.size, 1))

def plot(w1, b1, w2, b2, scale=1.):
    y_val = model(x_val, w1, b1, w2, b2)
    plt.plot(x_val, y_val * scale, color="orange")
    
    plt.scatter(clock, price * scale)
    plt.xlabel("clock")
    plt.ylabel("price")
    plt.ylim([(price *scale).min() - 0.15 * scale, (price * scale).max() + 0.15 * scale])

In [ ]:
# パラメータの初期値設定、モデル可視化
w1 = np.array([[-0.683, -0.833, -0.049]], dtype=np.float32)
b1 = np.array([0.499, 0.085, -0.654], dtype=np.float32)
w2 = np.array([[246071, 125136, 36801]], dtype=np.float32) / 100000.
b2 = np.array([116149], dtype=np.float32) / 100000.

plot(w1, b1, w2, b2);

In [ ]:
# コスト計算用の関数 (1)

# レコード数
n = float(price.size)

def calc_cost1(w1, b1, w2, b2):
    cost = 0.
    for x, t in zip(clock, price):
        y = model(x, w1, b1, w2, b2)
        cost += 0.5 * ((y - t) ** 2.)
        
    cost /= n
    return cost[0]

In [ ]:
# コスト計算用の関数 (2)

def calc_cost2(w1, b1, w2, b2):
    # 全てのデータポイントについての予測値 yを求める
    y = model(clock, w1, b1, w2, b2)
    
    # yを使ってコスト計算、結果を返却
    cost = 0.5 * np.mean((y - price) ** 2.)
    return cost

In [ ]:
# 実際の計算
w1 = np.array([[-0.683, -0.833, -0.049]], dtype=np.float32)
b1 = np.array([0.499, 0.085, -0.654], dtype=np.float32)
w2 = np.array([[246071, 125136, 36801]], dtype=np.float32) / 100000.
b2 = np.array([116149], dtype=np.float32) / 100000.

alpha = 0.3
history = list()

for i in range(50000):
    # 現状のコスト値を計算、記録
    # （calc_cost2が実装できていれば そちらへ切り替え）
    cost = calc_cost1(w1, b1, w2, b2)
    history.append((cost))
    
    # 進捗モニタリング
    if i % 1000 == 0:
        print("{:>6d}, cost: {:7.5f}".format(i, cost))

    # 実際の学習
    
    # == 2層目 ==
    # errorを出しておく
    y = model(clock, w1, b1, w2, b2)
    e2 = y - price
    
    # jを w2、bで微分したものをそれぞれ求める
    j_w2_deriv = e2 * layer1(clock, w1, b1)
    j_b2_deriv = e2
    
    # 新しいパラメータ w2_new、b2_newを計算
    w2_new = w2 - alpha * np.mean(j_w2_deriv, axis=0, keepdims=True)
    b2_new = b2 - alpha * np.mean(j_b2_deriv, axis=0)
    
    # == 1層目 ==
    # errorを出しておく
    h1 = layer1(clock, w1, b1)
    e1 = e2 * w1 * h1 * (1. - h1)
    
    # jを w1、b1で微分したものをそれぞれ求める
    j_w1_deriv = e1 * clock
    j_b1_deriv = e1
    
    # 新しいパラメータ w1_new、b1_newを計算
    w1_new = w1 - alpha * np.mean(j_w1_deriv, axis=0, keepdims=True)
    b1_new = b1 - alpha * np.mean(j_b1_deriv, axis=0)
    
    # 新旧パラメータをまとめて入れ替え
    w2 = w2_new
    b2 = b2_new
    w1 = w1_new
    b1 = b1_new

In [ ]:
# 結果として得られたモデルの可視化
plot(w1, b1, w2, b2, scale=100000)

In [ ]:
# 学習曲線
plt.plot([t for t in history])
plt.xlabel("iterations")
plt.ylabel("cost")

In [ ]:
# iteration 50,000回で動かせたら、、200,000回ではどのような結果になるか？

## keepdimsについて、補足

In [ ]:
# 行列を一つ準備
m = np.array([[1, 2, 3], [2, 2, 2]], dtype=np.float32)
m

In [ ]:
# 単純に和を取ると、全ての要素の合計、スカラー値
np.sum(m)

In [ ]:
# 行方向（タテ方向、axis=0）に和を取る、ということもできる
# 結果は 列方向の広がりだけ残り、行ベクトル
s1 = np.sum(m, axis=0)
s1

In [ ]:
s1.shape

In [ ]:
s1.ndim

In [ ]:
# axis=0で和を取るときに、行方向の軸を保っておきたいことがある
#   -> keepdims=True を指定
# 言い換えると： ndimは変えずに 指定した方向に和を取る
s2 = np.sum(m, axis=0, keepdims=True)
s2

In [ ]:
s2.shape

In [ ]:
s2.ndim